# Project Code

### ProjectCSGirls 2022

In [12]:
##import libraries for use throughout our code

from IPython.display import clear_output, display, update_display #IPython is the computing shell for our programming!

import smbus #temperature

#temperature and humidity
from bme280 import BME280
bme280 = BME280() 

import busio #air quality sensor
import board #air quality sensor, buzzer, and display

#Air quality sensor
from board import *
i2c = board.I2C() 

#Google Sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

#Adafruit
import adafruit_ccs811
from Adafruit_IO import Client, Feed, RequestError
from dotenv import load_dotenv
import os

import time #for general use

Google Sheets Initial Code - Kristen

In [ ]:
sheet_name = "ProjectCSGirls"   #Define the name of our Google Sheet

# use credentials to create a client that interacts with the Google Drive API *FIX THIS @ 2:30!!!*
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('projectcsgirls-345320-ec70b92cf651.json', scopes)
client = gspread.authorize(creds)

# Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).sheet1

Google Sheets Initial Code - Katie

In [13]:
sheet_name = "ProjectCSGirls"   #Define the name of our Google Sheet

# use credentials to create a client that interacts with the Google Drive API
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('secret.json', scopes)
client = gspread.authorize(creds)

# Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).sheet1

Testing each measurement individually

In [9]:
##temperature

## is it working?
## how to print??

def readTemp():
    # By default the address of LM75A is set to 0x48
    address = 0x48

    # Read I2C data and calculate temperature
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

In [4]:
## humidity
h = bme280.get_humidity()
print (h)

88.2681598298333


In [5]:
## CO2 and TVOC
ccs = adafruit_ccs811.CCS811(i2c)
print("CO2: ", ccs.eco2, "TVOC:", ccs.tvoc)

CO2:  0 TVOC: 0


Adafruit Connection - Kristen

In [ ]:
#Load login token from .env file
load_dotenv()

ADAFRUIT_IO_USERNAME = os.environ.get("ADAFRUIT_IO_USERNAME") #Adafruit username
ADAFRUIT_IO_KEY = os.environ.get("ADAFRUIT_IO_KEY") #Adafruit password

aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

Adafruit Connection - Katie
*Kristen - I put this in my .env file, this is probably important to put in there :)*

In [10]:
#Load login token from .env file
load_dotenv()

ADAFRUIT_IO_USERNAME = os.environ.get("ADAFRUIT_IO_USERNAME") #Adafruit username
ADAFRUIT_IO_KEY = os.environ.get("ADAFRUIT_IO_KEY") #Adafruit password

aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

Location - Kristen

In [ ]:
metadata = {'lat': 35.92948,
            'lon': -84.1313,
            'ele': 0,
            'created_at':None} 

Location - Katie

In [14]:
metadata = {"lat": 40.262726,
            "lon": -74.94198,
            "ele": 0,
            "created_at": None}

Reading Adafruit feeds - Katie

In [15]:
exterFeedB = aio.feeds("external")
humidFeedB = aio.feeds("humidity")
co2FeedB = aio.feeds("co2")
tvocFeedB = aio.feeds("tvoc")

Reading Adafruit feeds - Kristen
*Kristen - Do you want to make your Adafruit ext. feed the same name as mine? Then our feeds are the same and we can have 1 cell for reading the feeds :)*

In [ ]:
exterFeedM = aio.feeds("externaltemperature")
humidFeedM = aio.feeds("humidity")
co2FeedM = aio.feeds("co2")
tvocFeedM = aio.feesd("tvoc")

Main Loop 
*Kristen - Dr. Proffen said we can have one main loop with the same variable names, so I combined them :)*

In [16]:
while True:
    #define letter variables
    t = readTemp()
    e = t * (9./5.) +32 
    h = bme280.get_humidity()
    co2 = ccs.eco2
    tvoc = ccs.tvoc
    
    #send data to Adafruit feeds
    aio.send_data(exterFeedB.key, e, metadata)
    aio.send_data(humidFeedB.key, h, metadata)
    aio.send_data(co2FeedB.key, co2, metadata)
    aio.send_data(tvocFeedB.key, tvoc, metadata)
    
    #send and print data on the Google Sheet
    now=datetime.now()
    now_time = now.strftime("%H:%M:%S")
    now_day = now.strftime("%m/%d/%Y")
    row=[now_day, now_time, e, h, co2, tvoc]
    sheet.append_row(row)
    
    #print data in Jupyter Notebooks (below)
    print(e, ":°Fahrenheit|")
    print(h, ":% Humidity|")
    print(co2, ": PPM (CO2)|")
    print(tvoc, ":PPB (TVOC)|")
    print()
    
    #wait a certain amount of time
    time.sleep(60)

67.1 :°Fahrenheit|
88.2681598298333 :% Humidity|
424 : PPM (CO2)|
3 :PPB (TVOC)|



KeyboardInterrupt: 